## Creating a Kanji-to-Hiragana Model 

### Load ReazonSpeech Dataset

In [24]:
from datasets import load_dataset

In [25]:
# Load reazonspeech corpus dataset
ds = load_dataset("reazon-research/reazonspeech", trust_remote_code=True)
print("number of rows: ", ds["train"].num_rows)

number of rows:  5323


In [26]:
# Print some dataset
for i in range(10):
    print(f"row {i}: {ds['train'][i]['transcription']}")

row 0: これまたジミーさん
row 1: 今も相手にロンバルドのほうに肩口で握られてもすぐさま流れを切る引き込み返しに変えたと。
row 2: 僕はタクシーのことに関してはあんまりこう。
row 3: ［バーミヤンズオンエア獲得も疑惑浮上］
row 4: そしてもう一枚がこちら。
row 5: 何？
row 6: 積極的にお金を使うべきだと主張する政治家や省庁と支出を抑えたい財務省との間でせめぎ合いが続きます。
row 7: 今大会のボキの泳ぎ杉内さんはどう感じてらっしゃいますか？
row 8: そして得点を阻止するため相手はボールを発射！
row 9: やはりオウミ初の女性役員は夏目さんかな。


### Kanji-to-Hiragana via pykakasi

In [27]:
import pykakasi

In [28]:
# Initilize model
kks = pykakasi.kakasi()

# Load and convert string 
text = ds['train'][8]['transcription']
result = kks.convert(text)

In [29]:
# Print items
print("original text: ", text, "\n\n")
for item in result:
    print("{}: kana '{}', hiragana '{}', romaji: '{}'".format(item['orig'], item['kana'], item['hira'], item['hepburn']))

hiragana_sentence = ''.join(item['hira'] for item in result)
print("\n\nresult text: ", hiragana_sentence)

original text:  そして得点を阻止するため相手はボールを発射！ 


そして: kana 'ソシテ', hiragana 'そして', romaji: 'soshite'
得点: kana 'トクテン', hiragana 'とくてん', romaji: 'tokuten'
を: kana 'ヲ', hiragana 'を', romaji: 'wo'
阻止: kana 'ソシ', hiragana 'そし', romaji: 'soshi'
するため: kana 'スルタメ', hiragana 'するため', romaji: 'surutame'
相手: kana 'アイテ', hiragana 'あいて', romaji: 'aite'
は: kana 'ハ', hiragana 'は', romaji: 'ha'
ボール: kana 'ボール', hiragana 'ぼーる', romaji: 'booru'
を: kana 'ヲ', hiragana 'を', romaji: 'wo'
発射: kana 'ハッシャ', hiragana 'はっしゃ', romaji: 'hassha'
！: kana '！', hiragana '！', romaji: '!'


result text:  そしてとくてんをそしするためあいてはぼーるをはっしゃ！


### Utilizing CaboCha for Kanji-to-Phonetic conversion
Spent few hours trying to install CaboCha model, but had difficulties downloading packages. CaboCha had little to no support on WSL machine. 

### Utilizing MeCab for Kanji-to-Phonetic conversion


Install Dependencies:

```bash
apt update
apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
apt install -y swig
pip install mecab-python3

apt install -y git curl xz-utils
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
pwd
./bin/install-mecab-ipadic-neologd -n -p /path/to/custom/directory

In [30]:
import MeCab

In [31]:
# Use the NEologd dictionary
mecab = MeCab.Tagger('-r /dev/null -d /mnt/syno128/volume1/ryos17/ryota-sato-internship/venv/lib/python3.10/site-packages/mecab-ipadic-neologd') 

text = "ポケモンGO"
mecab.parse("")  # Prevent GC issues
node = mecab.parseToNode(text)

while node:
    word = node.surface  # Get the word
    pos = node.feature.split(",")[0]  # Get the part of speech
    print(f"{word} -> {pos}")
    node = node.next


 -> BOS/EOS
ポケモンGO -> 名詞
 -> BOS/EOS


In [32]:
# code from https://qiita.com/shimajiroxyz/items/81452afdedaed767f436
import MeCab
import jaconv

#記号を削除する関数
#(以下
#http://prpr.hatenablog.jp/entry/2016/11/23/Python%E3%81%A7%E5%85%A8%E8%A7%92%E3%83%BB%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%82%92%E3%81%BE%E3%81%A8%E3%82%81%E3%81%A6%E6%B6%88%E3%81%97%E5%8E%BB%E3%82%8B
#よりそのまま引用)
import unicodedata
import string

def format_text(text):
    text = unicodedata.normalize("NFKC", text)  # 全角記号をざっくり半角へ置換（でも不完全）

    # 記号を消し去るための魔法のテーブル作成
    table = str.maketrans("", "", string.punctuation  + "「」、。・")
    text = text.translate(table)

    return text
#(引用ここまで)

m = MeCab.Tagger('-r /dev/null -d /mnt/syno128/volume1/ryos17/ryota-sato-internship/venv/lib/python3.10/site-packages/mecab-ipadic-neologd') #形態素解析用objectの宣言

def getPronunciation(text):
    m_result = m.parse(text).splitlines() #mecabの解析結果の取得
    m_result = m_result[:-1] #最後の1行は不要な行なので除く

    pro = '' #発音文字列全体を格納する変数
    for v in m_result:
        if '\t' not in v: continue
        surface = v.split('\t')[0] #表層形
        p = v.split('\t')[1].split(',')[-1] #発音を取得したいとき
        #p = v.split('\t')[1].split(',')[-2] #ルビを取得したいとき
        #発音が取得できていないときsurfaceで代用
        if p == '*': p = surface
        pro += p
    
    pro = jaconv.hira2kata(pro) #ひらがなをカタカナに変換
    pro = format_text(pro) #余計な記号を削除

    return pro

In [33]:
# test code
text = '今日はよく寝ました'
print(text)
print(getPronunciation(text))

今日はよく寝ました
キョーワヨクネマシタ


### Updating ReazonSpeech corpus to include 'hiragana' and 'phonetic' column

In [34]:
from tqdm import tqdm

In [35]:
# Load dataset
ds = load_dataset("reazon-research/reazonspeech", trust_remote_code=True)
ds_len = ds["train"].num_rows

# Initilize Kanji-to-Hiragana model
kks = pykakasi.kakasi()

# Collect hiragana and phonetic conversions for all rows
hiragana_column = []
phonetic_column = []
for i in tqdm(range(ds_len)): 
    cur_text = ds['train'][i]['transcription']
    # Convert to hiragana
    result = kks.convert(cur_text)
    cur_hiragana = ''.join(item['hira'] for item in result)
    hiragana_column.append(cur_hiragana)
    # Convert to phonetic
    cur_phonetic = getPronunciation(cur_text)
    phonetic_column.append(cur_phonetic)

# Add the new 'hiragana' and 'phonetic' column to the dataset
ds['train'] = ds['train'].add_column("hiragana", hiragana_column)
ds['train'] = ds['train'].add_column("phonetic", phonetic_column)

100%|██████████| 5323/5323 [00:27<00:00, 195.72it/s]


In [36]:
# Verify first 10 row
for i in range(10):
    print(f"row {i}: {ds['train'][i]} \n")

row 0: {'name': '000/000734dcb35d6.flac', 'audio': {'path': '/home/ryos17/.cache/huggingface/datasets/downloads/extracted/c95cf990cfc8e23a0a848dba2e22a0d0c08e6667f3dfefff00640d3cf73287da/000/000734dcb35d6.flac', 'array': array([-0.01309204, -0.01068115, -0.006073  , ...,  0.00613403,
        0.00558472,  0.00674438]), 'sampling_rate': 16000}, 'transcription': 'これまたジミーさん', 'hiragana': 'これまたじみーさん', 'phonetic': 'コレマタジミーサン'} 

row 1: {'name': '000/0024ae5c517e7.flac', 'audio': {'path': '/home/ryos17/.cache/huggingface/datasets/downloads/extracted/c95cf990cfc8e23a0a848dba2e22a0d0c08e6667f3dfefff00640d3cf73287da/000/0024ae5c517e7.flac', 'array': array([-0.04589844, -0.0730896 , -0.09643555, ..., -0.00741577,
        0.00476074,  0.01593018]), 'sampling_rate': 16000}, 'transcription': '今も相手にロンバルドのほうに肩口で握られてもすぐさま流れを切る引き込み返しに変えたと。', 'hiragana': 'いまもあいてにろんばるどのほうにかたぐちでにぎられてもすぐさまながれをきるひきこみかえしにかえたと。', 'phonetic': 'イマモアイテニロンバルドノホーニカタグチデニギラレテモスグサマナガレヲキルヒキコミガエシニカエタト'} 

row 2: {'name': '000/002607eb83